# An introduction to JuMP

*Based on a tutorial from Los Alamos National Laboratory Grid Science Winter School, 2019. Adapted by Line Roald and Bainian Hao in January 2020 for the course "ECE/CS/ISyE 524 Introduction to Optimization" at University of Wisconsin - Madison, using Julia v1.3.1 and JuMP v0.20.1. Adapted in January 2021 for Julia v1.5.3. Further adapted in January 2023 for Julia 1.8.3*

Welcome! This tutorial will introduce you to the basics of the latest stable version of **JuMP**. If you haven't yet, work through the tutorial on Julia and do the exercises first.

**WARNING**: This tutorial covers material that will be discussed in class in the first couple of weeks. If something feels confusing, don't worry too much right now.

Some useful resources as you are learning JuMP are:
- [JuMP documentation](https://jump.dev/JuMP.jl/stable/)
- [Textbook: Julia Programming for Operations Research](http://www.chkwon.net/julia/)
- [the Discourse forum](https://discourse.julialang.org/c/domain/opt)

Before we start, install the following Julia packages (we'll explain more about what those packages are later!).

In [ ]:
using Pkg
Pkg.add("Ipopt")     
Pkg.add("GLPK")
Pkg.add("Interact")
Pkg.add("Plots")

Installing the packages typically takes a moment. If you are wondering when a cell is done running, you can check on the left hand side:

In [\*]:     The cell is still running

In [1]:      The cell is done (the number within the brackets will change).


If you want to check which packages you have installed, the following command will give you the full list and the version numbers.

In [ ]:
Pkg.status()

If you want to update your packages to a newer version, you can use the command `Pkg.update()`.

In [ ]:
Pkg.update()

# Building an optimization model

First, load the JuMP package into your current environment.

In [ ]:
using JuMP

Now you can start building your optimization model, which we will also refer to as a **JuMP model**!

Remember that there are three parts to all optimization problems:

1. Decisions (represented by decision variables, also called *optimization variables*)
2. Objective (the goal we want to achieve, which is expressed as a function of the decision variables)
3. Constraints (limitations that describe which choices are possible to make, also expressed as functions of the decision variables)

We will go through how to model these three parts using Julia and JuMP one by one. Let's start with defining decision variables:

In [ ]:
first_model = Model()
@variable(first_model, y >= 0)
@variable(first_model, 1 <= z <= 2)
first_model

### What's going on here?
`@variable(model, x)` does four things:
1. it adds an *optimization* variable $x$ to the model
2. it creates a *JuMP* variable that acts as a reference to the optimization variable in the model
3. it creates a *Julia* variable `x` that points to the JuMP variable
4. it stores a reference to the JuMP variable in the model with the name `:x`

Let's define a variable $x$ with a lower bound $1.414$.

In [ ]:
model = Model()
@variable(model, x >= 1.414)

In [ ]:
# x is a JuMP variable
typeof(x)

We can access the variable by using its name `:x`


In [ ]:
model[:x]

In [ ]:
# Let's find the lower bound!
JuMP.lower_bound(model[:x])

### Other ways to create variables

Sometimes, we need to create problems with MANY variables. Then it is useful to not have to create each variable separately. 

A useful feature is that we can create arrays of JuMP variables.

In [ ]:
model = Model()
@variable(model, x[i = 1:4] >= i)
x

The indices of the arrays don't have to be integers. They can be anything, like a string `"name"` or a symbol `:symbol_name`.

Let's create a variable that uses both number indices and symbols!

In [ ]:
model = Model()
@variable(model, x[i = 1:2, j = [:A, :B]] >= i)

println("Printing my optimization variable: ")
println()
println(x)
println()
println("The lower bound of the first element is ", JuMP.lower_bound(x[1,:A]))

Another example, with strings as names:

In [ ]:
model = Model()
@variable(model, x[i = 1:4, j = ["one", "two"]] >= i)
x

What if I want to add two variables with the same name?

It will give an error!

In [ ]:
model = Model()
@variable(model, x >= 1)
@variable(model, x >= 2)

### Quiz Question

What will be the output when you run this cell? Why?

In [ ]:
JuMP.lower_bound(x)

### Anonymous variables

So far, we've only added *named* variables. As we saw above, this works great until we want to add two variables with the same name! To work around this isse, we can add *anonymous* variables.

The syntax for this is `x = @variable(model, lower_bound = 1)`.

In [ ]:
model = Model()

x = @variable(model)
@show JuMP.has_lower_bound(x)

x = @variable(model, lower_bound = 1)
@show JuMP.lower_bound(x)

x = @variable(model, [i = 1:2], lower_bound = i)
@show JuMP.lower_bound.(x)

model

So what's the difference between the *named* and *anonymous* syntax? 

Well, `@variable(model, x >= 2)` is (roughly) equivalent to

In [ ]:
model = Model()
x = model[:x] = @variable(model, lower_bound = 2, base_name = "x")
model

In [ ]:
model = Model()
@variable(model, x >= 2)
model

### Binary and integer variables

By default, the decision variables in Julia are continuous variables. However, we can also create binary and integer variables as follows:

In [ ]:
model = Model()
@variable(model, x >= 1, Int)
@variable(model, y, Bin)
model

### Constraints

Now that we've seen how to create variables, let's look at **constraints**. 

Remember that constraints are limitations on the valid choices of decision variables (for example, the production of football and soccer trophies is limited by the available amount of wood). They may involve one or more decision variable, and be formulated as inequalities or equalities. 

Let's formulate the following constraints with decision variables $x\geq 0$ and $y \geq 0$:

$2x+y \leq 1$

$2x+y \geq 1$

$2x+y = 1$

Here is an example:

In [ ]:
model = Model()
@variable(model, x >= 0)
@variable(model, y >= 0)

@constraint(model, c_less_than, 2x + y <= 1)
@constraint(model, c_greater_than, 2x + y >= 1)
@constraint(model, c_equal_to, 2x + y == 1)

model

Similar to the optimization variables, we can access the constraints using their names:

In [ ]:
model[:c_equal_to]

We can also define anonymous constraints:

In [ ]:
@constraint(model, [i = 1:2], i * x <= y)

model

Small questions for you to think about: Consider the model which is desribes as an output from the cell above. Which constraints are the anonymous ones?

### Objective Functions

Now let's look at the last main part of the optimization model: The objective function.

Note two important aspects:
1. The objective is formulated as a function of the optimization problem.
2. We need to specify whether we want to maximize or minimize this function.

Minimization problem (i.e., minimizing the objective function):

In [ ]:
model = Model()
@variable(model, x >= 0)

@objective(model, Min, 2x + 1)

model

Maximization problem (i.e., maximizing the objective function):

In [ ]:
model = Model()
@variable(model, x <= 2)

@objective(model, Max, 2x + 1)

model

## DO IT YOURSELF!

Try to build the optimization model for the Top Brass example from class!

*Top Brass Trophy Company makes large championship trophies for youth athletic leagues. At the moment, they are planning production for fall sports: football and soccer. Each football trophy has a wood base, an engraved plaque, a large brass football on top, and returns 12 dollars in profit. Soccer trophies are similar except that a brass soccer ball is on top, and the unit profit is only 9 dollars. Since the football has an asymmetric shape, its base requires 4 board feet of wood; the soccer base requires only 2 board feet. At the moment there are 1000 brass footballs in stock, 1500 soccer balls, 1750 plaques, and 4800 board feet of wood. What trophies should be produced from these supplies to maximize total profit assuming that all that are made can be sold?*

(This is not a graded exercise)

In [ ]:
# Top Brass Optimization Model

TBmodel = Model()





# Solving a Model


Once we've formulated a model, the next step is to solve it. This requires a **solver**. 

We will talk about solvers in the class on Tuesday, January 28. If you do this tutorial before that and feel confused, you can stop here. Otherwise, think of a solver as an amazing piece of software that will mtake your optimization model as an input and magically give you the solution back! (In fact, formost of the class, this is how we will think about solvers anyways.)

JuMP supports lots of different solvers. The [JuMP documentation](http://www.juliaopt.org/JuMP.jl/v0.20/installation/#Getting-Solvers-1) contains a list of the supported solvers and the types of problems each solver supports.

For this tutorial, we're going to use two solvers in particular.

The first solver is the [GNU Linear Programming Kit (GLPK)](https://www.gnu.org/software/glpk/). This solver supports linear programs with continous variables.

GLPK is available via the [GLPK.jl](https://github.com/JuliaOpt/GLPK.jl) package.

The second solver is the COIN-OR [Interior Point OPTimizer (Ipopt)](https://projects.coin-or.org/Ipopt). This solver supports nonlinear programs with continous variables.

Ipopt is available via the [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) package.

In [ ]:
using GLPK
using Ipopt

There are two ways to add a solver to a JuMP model:

In [ ]:
model = Model(optimizer_with_attributes(GLPK.Optimizer))

# ... or ...

model = Model()
set_optimizer(model, optimizer_with_attributes(Ipopt.Optimizer))

Each solver can only handle certain types of problems. If you try to solve an unsupported problem type (such as using the linear solver GLPK for a nonlinear problem), an error will be thrown:

In [ ]:
model = Model(optimizer_with_attributes(GLPK.Optimizer))
@variable(model, 0 <= x <= π)
@NLobjective(model, Min, cos(x)^2) # NOTE: This is a non-linear objective function!
optimize!(model)

Let's try instead if the IpOpt solver, which is a solver for non-linear optimization problems.

In [ ]:
set_optimizer(model, optimizer_with_attributes(Ipopt.Optimizer))
optimize!(model)

## Getting solutions

- Use `objective_value(::Model)` to get the objective value
- Use `value(::VariableRef)` to get the value of a variable

In [ ]:
x_value = value(x)
obj_value = objective_value(model)

println("The optimal decision is ", x_value)
println("The objective value is ", obj_value)

## DO IT YOURSELF!

Try to solve the Top Brass model. What is the optimal objective function? How many football and soccer trophies should the company build?

Tip: Since you have already defined the optimization model `TBmodel` above, use the command `set_optimizer(model, optimizer_with_attributes(Ipopt.Optimizer)))` to determine the solver. 

Question: What do you think is the better choice, Ipopt or GLPK? Why?

## More advanced - to remember for later in class!

After solving a model, JuMP can report three different statuses:

- `termination_status(model)` explains why the solver stopped. Common statuses are `OPTIMAL`, `INFEASIBLE`, `DUAL_INFEASIBLE` (i.e., primal is potentially unbounded), and `LOCALLY_SOLVED`.

- `primal_status(model)` explains how to interpret the primal solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

- `dual_status(model)` explains how to interpret the dual solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

Information about both primal and dual variables are available:
- Use `value(::VariableRef)` to get the value of a primal variable
- Use `dual(::ConstraintRef)` to get the value of the dual variable associated with a constraint

In [ ]:
model = Model(optimizer_with_attributes(Ipopt.Optimizer))
@variable(model, 0 <= x <= π)
@NLobjective(model, Min, cos(x)^2) # NOTE: This is a non-linear objetive function!
optimize!(model)
x_value = value(x)
obj_value = objective_value(model)

println()
println("====== Let's look at the solution! =======")
println()
println("Termination status: ", termination_status(model))
println("Primal status:      ", primal_status(model))
println("Dual status:        ", dual_status(model))
println("      x | $(x_value)")
println("    π/2 | $(π/2)")
println("--------+----------------------")
println("cos²(x) | $(obj_value)")

## Interpreting statuses

After solving a model, JuMP can report three different statuses:

- `termination_status(model)` explains why the solver stopped. Common statuses are `OPTIMAL`, `INFEASIBLE`, `DUAL_INFEASIBLE` (i.e., primal is potentially unbounded), and `LOCALLY_SOLVED`.

- `primal_status(model)` explains how to interpret the primal solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

- `dual_status(model)` explains how to interpret the dual solution vector. Common statuses are `FEASIBLE_POINT` and `NO_SOLUTION`.

Information about both primal and dual variables are available:
- Use `value(::VariableRef)` to get the value of a primal variable
- Use `dual(::ConstraintRef)` to get the value of the dual variable associated with a constraint